# Image Projection

In [1]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
from pyquaternion.quaternion import Quaternion
from nuscenes.utils.geometry_utils import transform_matrix
import shutil


dataset_path = '/mnt/data/datasets/nuscenes/v1.0-mini/'
nusc = NuScenes(version='v1.0-mini', dataroot=dataset_path, verbose=True)

Loading NuScenes tables for version v1.0-mini...
Loading nuScenes-lidarseg...
32 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
404 lidarseg,
Done loading in 0.332 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [ ]:
nusc.sensor

In [2]:
#all_camera_tokens = ['725903f5b62f56118f4094b46a4470d8', 'ce89d4f3050b5892b33b3d328c5e82a3', 'a89643a5de885c6486df2232dc954da2',
#                     'ec4b5d41840a509984f7ec36419d4c09', '2f7ad058f1ac5557bf321c7543758f43', 'ca7dba2ec9f95951bbe67246f7f2c3f7']
#lidar_token = ['dc8b396651c05aedbb9cdaae573bb567']

all_cam_strings = ['CAM_FRONT','CAM_FRONT_RIGHT','CAM_BACK_RIGHT','CAM_BACK','CAM_BACK_LEFT','CAM_FRONT_LEFT']
lidar_string = ['LIDAR_TOP']

In [ ]:
calibrations = {}
for csensor in nusc.calibrated_sensor:
    token = csensor['sensor_token']
    #if token in all_camera_tokens:
    if token == '725903f5b62f56118f4094b46a4470d8':
        print(f"Found camera: {csensor['rotation']} and {csensor['translation']}")

In [11]:
print(f"number of scenes: {len(nusc.scene)}")
#for scene in nusc.scene:
scene = nusc.scene[3]
    
extrinsics = {}
intrinsics = {}
for cam_str in all_cam_strings:
    first_sample_token = scene['first_sample_token']
    sample = nusc.get('sample', first_sample_token)
    cam_data = nusc.get('sample_data', sample['data'][cam_str])
    sensor = nusc.get('calibrated_sensor', cam_data['calibrated_sensor_token'])
    T_B_C = transform_matrix(sensor["translation"], Quaternion(sensor["rotation"]))
    extrinsics[cam_str] = T_B_C
    intrinsics[cam_str] = sensor['camera_intrinsic']
    
print(f'Intrinsics: {intrinsics["CAM_FRONT"]}')
print(f'Exstrinsics: \n{extrinsics["CAM_FRONT"]}')

number of scenes: 10
Intrinsics: [[1252.8131021185304, 0.0, 826.588114781398], [0.0, 1252.8131021185304, 469.9846626224581], [0.0, 0.0, 1.0]]
Exstrinsics: 
[[ 0.01026021  0.00843345  0.9999118   1.72200568]
 [-0.99987258  0.01231626  0.01015593  0.00475453]
 [-0.01222952 -0.99988859  0.00855874  1.49491292]
 [ 0.          0.          0.          1.        ]]


In [24]:
scene = nusc.scene[3]

first_sample_token = scene['first_sample_token']
sample = nusc.get('sample', first_sample_token)
cam_data = nusc.get('sample_data', sample['data'][lidar_string[0]])
sensor = nusc.get('calibrated_sensor', cam_data['calibrated_sensor_token'])
T_E_C = transform_matrix(sensor["translation"], Quaternion(sensor["rotation"]))
extrinsics[lidar_string[0]] = T_E_C

print(f'Exstrinsics: \n{extrinsics[lidar_string[0]]}')

Exstrinsics: 
[[-5.42795004e-04  9.98930699e-01  4.62294677e-02  9.85793000e-01]
 [-9.99995492e-01 -4.05693167e-04 -2.97501151e-03  0.00000000e+00]
 [-2.95307535e-03 -4.62308742e-02  9.98926417e-01  1.84019000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [26]:
poserecord = nusc.get('ego_pose', cam_data['ego_pose_token'])
T_G_E = transform_matrix(poserecord["translation"], Quaternion(poserecord["rotation"]))
T_G_C = T_G_E * T_E_C
T_G_C

array([[-5.42648302e-04, -2.31549730e-02,  8.23043874e-05,
         1.74724322e+03],
       [-2.31188570e-02, -4.05416599e-04, -8.56309169e-05,
         0.00000000e+00],
       [ 7.22417630e-06,  1.32841879e-03,  9.98510950e-01,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [73]:
processed_folder = '/mnt/data/datasets/nuscenes/processed/'
for scene in nusc.scene:
    for cam_str in all_cam_strings:
        first_sample_token = scene['first_sample_token']
        sample = nusc.get('sample', first_sample_token)
        cam_data = nusc.get('sample_data', sample['data'][cam_str])

        out_dir = processed_folder + cam_str + '/'
        jpgfile = dataset_path + cam_data['filename']
        print(f"Copy {jpgfile} to {out_dir}")
        shutil.copy(jpgfile, out_dir)
        
    scene_name = scene['name']
    print(f'Processed scene {scene_name}\n')
print(f"Finished processing all scenes.")

Copy /mnt/data/datasets/nuscenes/v1.0-mini/samples/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402927612460.jpg to /mnt/data/datasets/nuscenes/processed/CAM_FRONT/
Copy /mnt/data/datasets/nuscenes/v1.0-mini/samples/CAM_FRONT_RIGHT/n015-2018-07-24-11-22-45+0800__CAM_FRONT_RIGHT__1532402927620339.jpg to /mnt/data/datasets/nuscenes/processed/CAM_FRONT_RIGHT/
Copy /mnt/data/datasets/nuscenes/v1.0-mini/samples/CAM_BACK_RIGHT/n015-2018-07-24-11-22-45+0800__CAM_BACK_RIGHT__1532402927627893.jpg to /mnt/data/datasets/nuscenes/processed/CAM_BACK_RIGHT/
Copy /mnt/data/datasets/nuscenes/v1.0-mini/samples/CAM_BACK/n015-2018-07-24-11-22-45+0800__CAM_BACK__1532402927637525.jpg to /mnt/data/datasets/nuscenes/processed/CAM_BACK/
Copy /mnt/data/datasets/nuscenes/v1.0-mini/samples/CAM_BACK_LEFT/n015-2018-07-24-11-22-45+0800__CAM_BACK_LEFT__1532402927647423.jpg to /mnt/data/datasets/nuscenes/processed/CAM_BACK_LEFT/
Copy /mnt/data/datasets/nuscenes/v1.0-mini/samples/CAM_FRONT_LEFT/n015-2018-07

'scene-1100'